In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import warnings
import os

warnings.filterwarnings('ignore')

# Add src to path
import sys
sys.path.append('..')

from src.config import (
    SEQUENCES_DIR, BEST_MODEL_PATH, CHECKPOINTS_DIR, TRAINING_FIGURES_DIR,
    LOGS_DIR, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN,
    ENCODER_HIDDEN_SIZE, ENCODER_NUM_LAYERS, ENCODER_DROPOUT, ENCODER_BIDIRECTIONAL,
    DECODER_HIDDEN_SIZE, DECODER_NUM_LAYERS, DECODER_DROPOUT,
    BATCH_SIZE, LEARNING_RATE, WEIGHT_DECAY, NUM_EPOCHS,
    EARLY_STOPPING_PATIENCE, GRADIENT_CLIP, TEACHER_FORCING_RATIO,
    LR_SCHEDULER, DEVICE, RANDOM_SEED, set_seed
)
from src.dataset import create_dataloaders
from src.model import build_model
from src.train import train
from src.utils import print_gpu_info, save_figure, load_json

# Set random seed
set_seed(RANDOM_SEED)

# Create logs directory
os.makedirs(LOGS_DIR, exist_ok=True)

print("Libraries imported successfully!")
print_gpu_info()

## 6.1 Load Sequences

In [ ]:
# Load sequences
X_train = np.load(os.path.join(SEQUENCES_DIR, 'X_train.npy'))
y_train = np.load(os.path.join(SEQUENCES_DIR, 'y_train.npy'))
X_val = np.load(os.path.join(SEQUENCES_DIR, 'X_val.npy'))
y_val = np.load(os.path.join(SEQUENCES_DIR, 'y_val.npy'))
X_test = np.load(os.path.join(SEQUENCES_DIR, 'X_test.npy'))
y_test = np.load(os.path.join(SEQUENCES_DIR, 'y_test.npy'))

# Load metadata
metadata = load_json(os.path.join(SEQUENCES_DIR, 'metadata.json'))

print("Data loaded:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

In [ ]:
# Get dimensions
n_features = X_train.shape[2]
print(f"\nNumber of input features: {n_features}")
print(f"Input sequence length: {INPUT_SEQ_LEN}")
print(f"Output sequence length: {OUTPUT_SEQ_LEN}")

## 6.2 Create DataLoaders

In [ ]:
# Create DataLoaders
train_loader, val_loader, test_loader = create_dataloaders(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test,
    batch_size=BATCH_SIZE,
    num_workers=0
)

print(f"\nTrain batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

## 6.3 Build Model

In [ ]:
# Build Encoder-Decoder model
model = build_model(
    input_size=n_features,
    hidden_size=ENCODER_HIDDEN_SIZE,
    num_layers=ENCODER_NUM_LAYERS,
    dropout=ENCODER_DROPOUT,
    bidirectional=ENCODER_BIDIRECTIONAL,
    output_seq_len=OUTPUT_SEQ_LEN,
    device=DEVICE
)

In [ ]:
# Model architecture summary
print("\nModel Architecture:")
print("=" * 60)
print(model)

## 6.4 Training Configuration

In [ ]:
# Print training configuration
print("Training Configuration:")
print("=" * 60)
print(f"Device: {DEVICE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Weight decay: {WEIGHT_DECAY}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Early stopping patience: {EARLY_STOPPING_PATIENCE}")
print(f"Gradient clipping: {GRADIENT_CLIP}")
print(f"Teacher forcing ratio: {TEACHER_FORCING_RATIO}")
print(f"LR scheduler: {LR_SCHEDULER}")
print("=" * 60)

## 6.4a Load Optuna Best Params (Optional)

Run this cell if you have already run Optuna optimization and want to use the best parameters.

In [ ]:
# Load best parameters from Optuna (if available)
best_params_path = os.path.join(LOGS_DIR, 'best_params.json')

USE_OPTUNA_PARAMS = False  # Set to True to use Optuna parameters

if USE_OPTUNA_PARAMS and os.path.exists(best_params_path):
    optuna_params = load_json(best_params_path)
    print("Loaded Optuna best parameters:")
    for k, v in optuna_params.items():
        print(f"  {k}: {v}")
    
    # Override config values
    ENCODER_HIDDEN_SIZE = optuna_params.get('hidden_size', ENCODER_HIDDEN_SIZE)
    ENCODER_NUM_LAYERS = optuna_params.get('num_layers', ENCODER_NUM_LAYERS)
    ENCODER_DROPOUT = optuna_params.get('dropout', ENCODER_DROPOUT)
    LEARNING_RATE = optuna_params.get('learning_rate', LEARNING_RATE)
    BATCH_SIZE = optuna_params.get('batch_size', BATCH_SIZE)
    WEIGHT_DECAY = optuna_params.get('weight_decay', WEIGHT_DECAY)
    TEACHER_FORCING_RATIO = optuna_params.get('teacher_forcing_ratio', TEACHER_FORCING_RATIO)
    
    print("\nConfig values updated!")
else:
    print("Using default config parameters")

## 6.5 Train Model

In [ ]:
# Train model with logging
history = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    teacher_forcing_ratio=TEACHER_FORCING_RATIO,
    gradient_clip=GRADIENT_CLIP,
    early_stopping_patience=EARLY_STOPPING_PATIENCE,
    lr_scheduler_type=LR_SCHEDULER,
    checkpoint_dir=CHECKPOINTS_DIR,
    best_model_path=BEST_MODEL_PATH,
    device=DEVICE,
    log_dir=LOGS_DIR  # Enable logging
)

## 6.6 Training Visualization

In [ ]:
# Plot learning curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
epochs = range(1, len(history['train_loss']) + 1)
axes[0].plot(epochs, history['train_loss'], 'b-', label='Train Loss')
axes[0].plot(epochs, history['val_loss'], 'r-', label='Validation Loss')
axes[0].axvline(x=history['best_epoch'] + 1, color='green', linestyle='--', label=f'Best Epoch ({history["best_epoch"]+1})')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True)

# Learning rate
axes[1].plot(epochs, history['learning_rate'], 'g-')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Learning Rate')
axes[1].set_title('Learning Rate Schedule')
axes[1].set_yscale('log')
axes[1].grid(True)

plt.tight_layout()
os.makedirs(TRAINING_FIGURES_DIR, exist_ok=True)
save_figure(fig, os.path.join(TRAINING_FIGURES_DIR, 'learning_curves.png'))
plt.show()

In [ ]:
# Training summary
print("\n" + "=" * 60)
print("TRAINING SUMMARY")
print("=" * 60)
print(f"Total epochs trained: {len(history['train_loss'])}")
print(f"Best epoch: {history['best_epoch'] + 1}")
print(f"Best validation loss: {history['best_val_loss']:.6f}")
print(f"Final training loss: {history['train_loss'][-1]:.6f}")
print(f"Final validation loss: {history['val_loss'][-1]:.6f}")
print(f"Model saved to: {BEST_MODEL_PATH}")
print("=" * 60)

## 6.7 Save Training History

In [ ]:
# Save training history
from src.utils import save_json

history_to_save = {
    'train_loss': [float(x) for x in history['train_loss']],
    'val_loss': [float(x) for x in history['val_loss']],
    'learning_rate': [float(x) for x in history['learning_rate']],
    'best_epoch': int(history['best_epoch']),
    'best_val_loss': float(history['best_val_loss'])
}

save_json(history_to_save, os.path.join(TRAINING_FIGURES_DIR, 'training_history.json'))
print("Training history saved!")

## Summary

**Model Training completed:**
1. ✅ Loaded sequence data
2. ✅ Created DataLoaders
3. ✅ Built Encoder-Decoder model
4. ✅ Trained with GPU
5. ✅ Early stopping applied
6. ✅ Saved best model
7. ✅ Visualized learning curves

**Next step:** Evaluation (07_Evaluation.ipynb)